# RNN

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import json
import matplotlib.pyplot as plt

import os

In [ ]:
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.15.0


In [ ]:
# Replace 'your_folder_path' with the actual path to your folder containing text files
folder_path = './drive/MyDrive/DatosInvestigacion/resultados/energy/data-continua/'

In [ ]:
file_path = os.path.join(folder_path, 'df_clean_all_2_22.csv')
df = pd.read_csv(file_path, index_col=0, parse_dates=True)

In [ ]:
df_compare = df.resample('2T').asfreq()

In [ ]:
df.isna().sum()

Potencia_3F_[W]           0
Potencia_3F_[VA]          0
Factor de Potencia        0
Voltaje_(R)_[V]           0
Voltaje_(S)_[V]           0
Voltaje_(T)_[V]           0
Voltaje_(RS)_[V]          0
Voltaje_(ST)_[V]          0
Voltaje_(TR)_[V]          0
Corriente_R_[A]           0
Corriente_S_[A]           0
Corriente_T_[A]           0
Potencia_R_[VA]           0
Potencia_S_[VA]           0
Potencia_T_[VA]           0
Potencia_R_[W]            0
Potencia_S_[W]            0
Potencia_T_[W]            0
Potencia_R_[VAR]          0
Potencia_S_[VAR]          0
Potencia_T_[VAR]          0
Corriente_N_[A]           0
Frecuencia_[Hz]           0
Potencia_3F_[VAR]         0
suma_potencias_W          0
suma_potencias_VA         0
factor_total_calculado    0
anomaly                   0
dtype: int64

In [ ]:
df_compare.isna().sum()

Potencia_3F_[W]           50
Potencia_3F_[VA]          50
Factor de Potencia        50
Voltaje_(R)_[V]           50
Voltaje_(S)_[V]           50
Voltaje_(T)_[V]           50
Voltaje_(RS)_[V]          50
Voltaje_(ST)_[V]          50
Voltaje_(TR)_[V]          50
Corriente_R_[A]           50
Corriente_S_[A]           50
Corriente_T_[A]           50
Potencia_R_[VA]           50
Potencia_S_[VA]           50
Potencia_T_[VA]           50
Potencia_R_[W]            50
Potencia_S_[W]            50
Potencia_T_[W]            50
Potencia_R_[VAR]          50
Potencia_S_[VAR]          50
Potencia_T_[VAR]          50
Corriente_N_[A]           50
Frecuencia_[Hz]           50
Potencia_3F_[VAR]         50
suma_potencias_W          50
suma_potencias_VA         50
factor_total_calculado    50
anomaly                   50
dtype: int64

In [ ]:
df.sort_index(ascending=True, inplace=True)

In [ ]:
train_size = int(len(df)*0.7)
train_size

7332

In [ ]:
val_size = int(len(df)*0.1)
val_size

1047

In [ ]:
df_train = df.iloc[:train_size]

In [ ]:
df_val = df.iloc[train_size:train_size+val_size]

In [ ]:
len(df_val)

1047

In [ ]:
df_test = df.iloc[train_size+val_size:]

In [ ]:
df_train.head(1)

,Potencia_3F_[W],Potencia_3F_[VA],Factor de Potencia,Voltaje_(R)_[V],Voltaje_(S)_[V],Voltaje_(T)_[V],Voltaje_(RS)_[V],Voltaje_(ST)_[V],Voltaje_(TR)_[V],Corriente_R_[A],...,Potencia_R_[VAR],Potencia_S_[VAR],Potencia_T_[VAR],Corriente_N_[A],Frecuencia_[Hz],Potencia_3F_[VAR],suma_potencias_W,suma_potencias_VA,factor_total_calculado,anomaly
2020-12-17 09:10:00,764.273,870.925,0.877,126.198,125.841,126.782,217.997,219.529,218.545,1.436,...,-3.085,88.859,-68.702,2.926,59.983,17.073,764.273,870.925,0.877542,1


In [ ]:
df_train.tail(1)

,Potencia_3F_[W],Potencia_3F_[VA],Factor de Potencia,Voltaje_(R)_[V],Voltaje_(S)_[V],Voltaje_(T)_[V],Voltaje_(RS)_[V],Voltaje_(ST)_[V],Voltaje_(TR)_[V],Corriente_R_[A],...,Potencia_R_[VAR],Potencia_S_[VAR],Potencia_T_[VAR],Corriente_N_[A],Frecuencia_[Hz],Potencia_3F_[VAR],suma_potencias_W,suma_potencias_VA,factor_total_calculado,anomaly
2020-12-27 14:44:00,453.005,532.047,0.826,126.302,127.01,128.644,218.619,222.142,220.757,1.278,...,-5.111,61.525,-37.721,1.941,58.233,18.693,464.086,548.099,0.846719,1


In [ ]:
df_val.head(1)

,Potencia_3F_[W],Potencia_3F_[VA],Factor de Potencia,Voltaje_(R)_[V],Voltaje_(S)_[V],Voltaje_(T)_[V],Voltaje_(RS)_[V],Voltaje_(ST)_[V],Voltaje_(TR)_[V],Corriente_R_[A],...,Potencia_R_[VAR],Potencia_S_[VAR],Potencia_T_[VAR],Corriente_N_[A],Frecuencia_[Hz],Potencia_3F_[VAR],suma_potencias_W,suma_potencias_VA,factor_total_calculado,anomaly
2020-12-27 14:46:00,466.538,548.037,0.851,126.326,127.026,128.542,218.641,222.136,220.63,1.276,...,-5.289,63.24,-38.924,2.007,60.006,19.027,466.537,548.038,0.851286,1


In [ ]:
df_val.tail(1)

,Potencia_3F_[W],Potencia_3F_[VA],Factor de Potencia,Voltaje_(R)_[V],Voltaje_(S)_[V],Voltaje_(T)_[V],Voltaje_(RS)_[V],Voltaje_(ST)_[V],Voltaje_(TR)_[V],Corriente_R_[A],...,Potencia_R_[VAR],Potencia_S_[VAR],Potencia_T_[VAR],Corriente_N_[A],Frecuencia_[Hz],Potencia_3F_[VAR],suma_potencias_W,suma_potencias_VA,factor_total_calculado,anomaly
2020-12-29 01:58:00,706.209,816.121,0.865,128.061,127.643,129.006,220.783,222.887,222.623,1.04,...,-22.272,53.329,-158.103,1.62,59.987,-127.046,706.21,816.122,0.865324,1


In [ ]:
df_test.head(1)

,Potencia_3F_[W],Potencia_3F_[VA],Factor de Potencia,Voltaje_(R)_[V],Voltaje_(S)_[V],Voltaje_(T)_[V],Voltaje_(RS)_[V],Voltaje_(ST)_[V],Voltaje_(TR)_[V],Corriente_R_[A],...,Potencia_R_[VAR],Potencia_S_[VAR],Potencia_T_[VAR],Corriente_N_[A],Frecuencia_[Hz],Potencia_3F_[VAR],suma_potencias_W,suma_potencias_VA,factor_total_calculado,anomaly
2020-12-29 02:00:00,656.165,758.895,0.865,128.043,127.538,128.959,220.663,222.8,222.536,1.043,...,-22.357,36.646,-147.48,1.6,59.985,-133.19,656.164,758.894,0.864632,1


In [ ]:
df_test.tail(1)

,Potencia_3F_[W],Potencia_3F_[VA],Factor de Potencia,Voltaje_(R)_[V],Voltaje_(S)_[V],Voltaje_(T)_[V],Voltaje_(RS)_[V],Voltaje_(ST)_[V],Voltaje_(TR)_[V],Corriente_R_[A],...,Potencia_R_[VAR],Potencia_S_[VAR],Potencia_T_[VAR],Corriente_N_[A],Frecuencia_[Hz],Potencia_3F_[VAR],suma_potencias_W,suma_potencias_VA,factor_total_calculado,anomaly
2020-12-31 23:58:00,702.121,811.937,0.839,127.734,128.241,129.285,220.938,223.671,222.634,1.07,...,-21.609,60.951,-154.166,1.584,58.249,-114.823,712.963,836.548,0.852268,1


In [ ]:
input_sets={
    '3F_W':['Potencia_3F_[W]','Corriente_T_[A]','Potencia_T_[VA]'],
    '3F_VA':['Potencia_3F_[VA]','Corriente_T_[A]','Potencia_T_[VA]'],
    '3F_factor':['Factor de Potencia','Potencia_R_[W]']
}

In [ ]:
input_sets.keys()

dict_keys(['3F_W', '3F_VA', '3F_factor'])

In [ ]:
ventana_prediccion =[30,15,1]
factor_ventana_entrada = [0.5,1,2,4]
num_neuronas_set = [50,100,150]
batch_size_set = [50,100,150]
num_layers_set = [1,2,3]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation, SimpleRNN
from tensorflow import keras
import tensorflow as tf

In [ ]:
def create_Model(n_input, LSTM_neurons = 100, layers=3,n_features=0,n_output=1):
  # define model
  model = Sequential()
  if(layers>1):
    model.add(SimpleRNN(LSTM_neurons, activation='relu', input_shape=(n_input, n_features+1),return_sequences=True))
    for i in range(layers-2):
        model.add(SimpleRNN(LSTM_neurons, activation='relu',return_sequences=True))
    model.add(SimpleRNN(LSTM_neurons, activation='relu'))
  else:
    model.add(SimpleRNN(LSTM_neurons, activation='relu', input_shape=(n_input, n_features+1)))
  model.add(Dense(n_output))
  model.compile(optimizer='adam', loss='mse')
  return model

In [ ]:
def get_XY(scaled_data,past_hours,prediction_window):
  # Prepare the dataset with lagged features
  X, y = [], []
  n = past_hours
  pw = prediction_window
  for i in range(n, len(scaled_data)-pw):
      X.append(scaled_data[i-n:i, :])  # Use predictor variables for each timestep
      y.append(scaled_data[i:i+pw, 0])  # X0 is the target variable
  X_train = np.array(X)
  y_train = np.array(y)
  X_train_flattened = X_train#.reshape(X_train.shape[0], -1)
  y_train_flattened = y_train.reshape(y_train.shape[0], -1)
  return X_train_flattened, y_train_flattened

In [ ]:
combinaciones_anteriores = []

In [ ]:
gsearch_path = os.path.join(folder_path, f'investigacion/grid_search2/GS_RNN_m_3F_W.txt')
with open(gsearch_path,'r') as fp:
    for line in fp:
      combinaciones_anteriores.append(json.loads(line))

In [ ]:
gsearch_path = os.path.join(folder_path, f'investigacion/grid_search2/GS_RNN_m_3F_VA.txt')
with open(gsearch_path,'r') as fp:
    for line in fp:
      combinaciones_anteriores.append(json.loads(line))

In [ ]:
gsearch_path = os.path.join(folder_path, f'investigacion/grid_search2/GS_RNN_m_3F_factor.txt')
with open(gsearch_path,'r') as fp:
    for line in fp:
      combinaciones_anteriores.append(json.loads(line))

In [ ]:
combinaciones_anteriores[-2:]

[{'Model': 'RNN',
  'RMSE': 0.015878545095091828,
  'Fuente': '3F_factor',
  'n_input': 4,
  'n_output': 1,
  'num_neuronas': 150,
  'batch_size': 100,
  'num_layers': 3,
  'MSE': 0.00025212819433686475},
 {'Model': 'RNN',
  'RMSE': 0.016722832610233093,
  'Fuente': '3F_factor',
  'n_input': 4,
  'n_output': 1,
  'num_neuronas': 150,
  'batch_size': 150,
  'num_layers': 3,
  'MSE': 0.00027965313050987535}]

In [ ]:
min_error = 1000000

In [ ]:
string_list = []

In [ ]:
last_result = combinaciones_anteriores[-1]

In [ ]:
for result in combinaciones_anteriores:
  current_string = ''
  for key in result.keys():
    if 'MSE' not in key:
      current_string+=str(result[key])
  string_list.append(current_string)
  if (result['RMSE']<min_error and
    result['n_output']==last_result['n_output'] and
    result['Fuente']==last_result['Fuente']):
    min_error = result['RMSE']

In [ ]:
string_list[-5:]

['RNN3F_factor411501002',
 'RNN3F_factor411501502',
 'RNN3F_factor41150503',
 'RNN3F_factor411501003',
 'RNN3F_factor411501503']

In [ ]:
min_error

0.014224265835393296

In [ ]:
result.keys()

dict_keys(['Model', 'RMSE', 'Fuente', 'n_input', 'n_output', 'num_neuronas', 'batch_size', 'num_layers', 'MSE'])

In [ ]:
last_n_output=last_result['n_output']

In [ ]:
last_min_err = min_error

In [ ]:
last_min_err

0.014224265835393296

In [ ]:
last_iset = last_result['Fuente']

In [ ]:
for input_set in input_sets.keys():
  df_train_i = df_train[input_sets[input_set]].copy()
  df_val_i = df_val[input_sets[input_set]].copy()
  # scaled data
  scaler.fit(df_train_i)
  scaled_train = scaler.transform(df_train_i)
  scaled_val = scaler.transform(df_val_i)
  # prediction window
  for pw in ventana_prediccion:
    n_output = pw
    if(last_n_output != n_output or input_set != last_iset):
      min_error = 100000 # hay que mejorar
    else:
      min_error = last_min_err

    for fi in factor_ventana_entrada:
      n_input = int(fi*pw)

      if(n_input<1):
        continue

      n_features = len(df_train_i.columns)-1
      for num_neuronas in num_neuronas_set:
        for num_layers in num_layers_set:
          for batch_size in batch_size_set:
            if('RNN'+str(input_set)+str(n_input)+str(n_output)+str(num_neuronas)+str(batch_size)+str(num_layers)
               in string_list):
              continue
            print(f'Current options: {input_set}/{n_input}/{n_output}/{num_neuronas}/{batch_size}/{num_layers}/best:{min_error}')
            X_train, y_train = get_XY(scaled_train,past_hours=n_input,prediction_window=n_output)
            model = create_Model(n_input,LSTM_neurons=num_neuronas,layers=num_layers,n_features=n_features,n_output=n_output)
            model.fit(X_train,y_train,epochs=10,verbose=False,batch_size=batch_size)

            X_val, y_val = get_XY(scaled_val,past_hours=n_input,prediction_window=n_output)
            predictions = model.predict(X_val,verbose=False)

            flatten_y = y_val.flatten()
            flatten_pred = predictions.flatten()

            true_y = np.zeros(shape=(len(flatten_pred), 1+n_features))
            true_y[:,0] = flatten_y
            true_y = scaler.inverse_transform(true_y)
            true_pred = np.zeros(shape=(len(flatten_pred), 1+n_features))
            true_pred[:,0] = flatten_pred
            true_pred = scaler.inverse_transform(true_pred)
            errorMSE = mean_squared_error(true_y[:,0],true_pred[:,0])
            errorRMSE = np.sqrt(errorMSE)
            if (errorRMSE < min_error):
              min_error = errorRMSE
              last_min_err = min_error
              print('BEST')
              model_path = os.path.join(folder_path, f'investigacion/grid_search2/RNN_m_model_{input_set}_pw{n_output}_iw{n_input}.h5')
              model.save(model_path)
            resultado = {
                'Model':'RNN',
                'RMSE':errorRMSE,
                'Fuente':input_set,
                'n_input':n_input,
                'n_output':n_output,
                'num_neuronas':num_neuronas,
                'batch_size':batch_size,
                'num_layers': num_layers,
                'MSE':errorMSE,
            }
            print(resultado)
            gsearch_path = os.path.join(folder_path, f'investigacion/grid_search2/GS_RNN_m_{input_set}.txt')
            with open(gsearch_path,"a") as file:
              resultado = json.dumps(resultado)
              file.write(resultado)
              file.write('\n')
